In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import plotly.express as px

import glob

from locations import *
from parsing import *

In [2]:
# Idea:
# Más adelante, agrupar las columnas desechadas en una sola para crear una sola llamada 'Descripción'
# y hacer un Cluster con NLP con ella

In [3]:
df = pd.read_csv('madrid_parsed.csv')
df

,price,title,location,lat,lng,agency,updated,Lavadero,Soleado,Balcón,...,Gas,No se aceptan mascotas,Consumo,Luz,Tipo de casa,Calle asfaltada,Emisiones,Sistema de seguridad,Calefacción,Urbanizado
0,176000.0,Apartamento en venta en Abantos-Monte Carmelo-...,Abantos-Monte Carmelo-El Rosario (San Lorenzo ...,40.593445,-4.145386,Inmuebles de SYMBOLIC GESTION INMOBILIARIA,1.674861e+09,Con acceso desde la cocina,Soleado,Desde el salón,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,116500.0,"Apartamento en venta en Calle de Ercilla, 20, ...","Calle de Ercilla, 20, cerca de Calle del Labra...",40.402079,-3.702151,Inmuebles de Inmobiliaria Grupo Habitazone,1.674947e+09,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,169000.0,Apartamento en venta en Ajalvir,Ajalvir,40.534457,-3.479415,Inmuebles de INMOBILIARIA TECNICA NORTE,1.673392e+09,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,174000.0,Apartamento en venta en Alcorcón,Casco Antiguo (Alcorcón),40.344358,-3.825283,Inmuebles de Elite Asesoramiento Inmobiliario SL,1.672960e+09,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,130000.0,Apartamento en venta en Alcorcón,Casco Antiguo (Alcorcón),40.347096,-3.827826,Inmuebles de ELEPHANT REAL ESTATE,1.672615e+09,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Eléctrica,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15841,135000.0,Piso en venta en Calle de la Circunvalación,Calle de la Circunvalación. Zona Suroeste (Tor...,40.454606,-3.455234,Inmuebles de ANGEL & RICARDO INMOBILIARIA,1.672960e+09,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15842,210000.0,Piso en venta en Calle de Urano,Calle de Urano. Zona Suroeste (Torrejón de Ardoz),40.441928,-3.473036,Inmuebles de Look & Find Retiro,1.672615e+09,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15843,324600.0,Piso en venta en Fresnos,Zona Suroeste (Torrejón de Ardoz),40.448403,-3.470899,Inmuebles de Cassas.es asesoria inmobiliaria,1.672874e+09,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Gas natural,NaN
15844,249900.0,Piso en venta en Calle de Cibeles,Calle de Cibeles. Zona Suroeste (Torrejón de A...,40.444707,-3.473520,Inmuebles de AS Fresnos Inmobiliaria,1.672874e+09,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Gas natural,NaN


In [4]:
fig = px.bar([df[x].dropna().count() for x in df.columns],
             y = df.columns,
             x = [df[x].dropna().count() for x in df.columns])

fig.update_layout(height = 1000)

In [5]:
# Lista de columnas a revisar para decidir cómo tratar la información para entrenar el modelo en el futuro
scratched_features_col = ['Cocina', 'Cocina equipada', 'Superficie solar', 'Sistema de seguridad', 'Portero automático',
                      'Aire acondicionado', 'Puerta blindada', 'Amueblado', 'Trastero', 'Calefacción', 'Gas',
                      'Tipo de casa', 'Piscina', 'Vidrios dobles', 'Planta', 'Tipo suelo', 'Balcón', 'Orientación',
                      'Armarios empotrados', 'Comedor']

# Cocina -> hacerla categórica (americana, independiente, amueblada, sin amueblar)
# Cocina equipada -> revisarla con 'Cocina'
# Superficie solar -> Seguramente sea interesante solo para chalets
# Sistema de seguridad -> hacerla categórica (conserje, alarma, videovigilancia, etc.) mezclándola con 'Portero automático'
# Portero automático -> relacionar con 'Sistema de seguridad'
# Aire acondicionado -> hacerla categórica numérica
# Puerta blindada -> hacerla binaria
# Amueblado -> hacerla categórica (sí, no, parcial)
# Trastero -> hacerla categórica (sí, no, comunitario)
# Calefacción -> hacerla categórica (gas natural, butano, eléctrico, frío-calor, chimenea, no)
# Gas -> hacerla categórica (revisar Calefacción)
# Tipo de casa -> hacerla categórica ('Adosada', 'Pareada', ' Rústica', ' Unifamiliar', 'otro)
# Piscina -> hacerla categórica (privada, comunitaria, opcional-comunitaria, no, etc)
# Vidrios dobles -> hacerla categórica (corredera, oscilobatiente, abrir-cerrar, doble aluminio, no)
# Planta -> hacerla categórica numérica (-2, -1, 0, 1, 2, etc)
# Tipo suelo -> hacerla binaria (frío, caliente)
# Balcón -> hacerla categórica numérica
# Armarios empotrados -> hacerla binaria
# Comedor -> hacerla categórica

In [6]:
features_col = ['price', 'title', 'lat', 'lng', 'location', 'Habitaciones', 'Superficie construida',
                'Jardín', 'Gastos de comunidad', 'Antigüedad', 'Baños', 'Superficie útil', 'Ascensor',
                'Garaje', 'Conservación', 'agency', 'updated']

# Garaje -> hacer varias columnas columnas (tien o no tiene, cantidad, metros cuadrados)
# location -> sacar columnas (tipo de vía, locación, etc)
# title -> revisar con columna 'Tipo de casa' para obtener una columna con tipo de vivienda

In [7]:
df_features = df[features_col]
df_features

,price,title,lat,lng,location,Habitaciones,Superficie construida,Jardín,Gastos de comunidad,Antigüedad,Baños,Superficie útil,Ascensor,Garaje,Conservación,agency,updated
0,176000.0,Apartamento en venta en Abantos-Monte Carmelo-...,40.593445,-4.145386,Abantos-Monte Carmelo-El Rosario (San Lorenzo ...,1.0,63 m²,Comunitario,Entre 40 y 60€,Entre 10 y 20 años,1.0,46 m²,Ascensor,1,A estrenar,Inmuebles de SYMBOLIC GESTION INMOBILIARIA,1.674861e+09
1,116500.0,"Apartamento en venta en Calle de Ercilla, 20, ...",40.402079,-3.702151,"Calle de Ercilla, 20, cerca de Calle del Labra...",NaN,25 m²,NaN,NaN,Más de 50 años,1.0,NaN,Ascensor,NaN,NaN,Inmuebles de Inmobiliaria Grupo Habitazone,1.674947e+09
2,169000.0,Apartamento en venta en Ajalvir,40.534457,-3.479415,Ajalvir,1.0,70 m²,NaN,NaN,NaN,1.0,67 m²,Ascensor,NaN,En buen estado,Inmuebles de INMOBILIARIA TECNICA NORTE,1.673392e+09
3,174000.0,Apartamento en venta en Alcorcón,40.344358,-3.825283,Casco Antiguo (Alcorcón),1.0,60 m²,NaN,NaN,NaN,1.0,50 m²,Ascensor,NaN,NaN,Inmuebles de Elite Asesoramiento Inmobiliario SL,1.672960e+09
4,130000.0,Apartamento en venta en Alcorcón,40.347096,-3.827826,Casco Antiguo (Alcorcón),2.0,60 m²,NaN,Entre 60 y 80€,NaN,1.0,54 m²,Ascensor,NaN,NaN,Inmuebles de ELEPHANT REAL ESTATE,1.672615e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15841,135000.0,Piso en venta en Calle de la Circunvalación,40.454606,-3.455234,Calle de la Circunvalación. Zona Suroeste (Tor...,3.0,67 m²,NaN,NaN,NaN,1.0,62 m²,Ascensor,NaN,A reformar,Inmuebles de ANGEL & RICARDO INMOBILIARIA,1.672960e+09
15842,210000.0,Piso en venta en Calle de Urano,40.441928,-3.473036,Calle de Urano. Zona Suroeste (Torrejón de Ardoz),1.0,79 m²,NaN,Entre 80 y 100€,NaN,2.0,63 m²,Ascensor,1,En buen estado,Inmuebles de Look & Find Retiro,1.672615e+09
15843,324600.0,Piso en venta en Fresnos,40.448403,-3.470899,Zona Suroeste (Torrejón de Ardoz),4.0,140 m²,NaN,NaN,NaN,3.0,NaN,Ascensor,1,NaN,Inmuebles de Cassas.es asesoria inmobiliaria,1.672874e+09
15844,249900.0,Piso en venta en Calle de Cibeles,40.444707,-3.473520,Calle de Cibeles. Zona Suroeste (Torrejón de A...,3.0,106 m²,NaN,Entre 80 y 100€,Entre 20 y 30 años,2.0,96 m²,NaN,1,En buen estado,Inmuebles de AS Fresnos Inmobiliaria,1.672874e+09


In [8]:
# Sería interesante poner una columna que tenga la fecha actual? Para que el modelo compare con la fecha de publicación

In [9]:
df_ = getType(df_features)


c:\Users\Aa\Documents\GitHub\pisoscom\parsing.py:178: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [10]:
df_

,price,title,lat,lng,location,Habitaciones,Superficie construida,Jardín,Gastos de comunidad,Antigüedad,Baños,Superficie útil,Ascensor,Garaje,Conservación,agency,updated,type
0,176000.0,Apartamento en venta en Abantos-Monte Carmelo-...,40.593445,-4.145386,Abantos-Monte Carmelo-El Rosario (San Lorenzo ...,1.0,63 m²,Comunitario,Entre 40 y 60€,Entre 10 y 20 años,1.0,46 m²,Ascensor,1,A estrenar,Inmuebles de SYMBOLIC GESTION INMOBILIARIA,1.674861e+09,Apartamento
1,116500.0,"Apartamento en venta en Calle de Ercilla, 20, ...",40.402079,-3.702151,"Calle de Ercilla, 20, cerca de Calle del Labra...",NaN,25 m²,NaN,NaN,Más de 50 años,1.0,NaN,Ascensor,NaN,NaN,Inmuebles de Inmobiliaria Grupo Habitazone,1.674947e+09,Apartamento
2,169000.0,Apartamento en venta en Ajalvir,40.534457,-3.479415,Ajalvir,1.0,70 m²,NaN,NaN,NaN,1.0,67 m²,Ascensor,NaN,En buen estado,Inmuebles de INMOBILIARIA TECNICA NORTE,1.673392e+09,Apartamento
3,174000.0,Apartamento en venta en Alcorcón,40.344358,-3.825283,Casco Antiguo (Alcorcón),1.0,60 m²,NaN,NaN,NaN,1.0,50 m²,Ascensor,NaN,NaN,Inmuebles de Elite Asesoramiento Inmobiliario SL,1.672960e+09,Apartamento
4,130000.0,Apartamento en venta en Alcorcón,40.347096,-3.827826,Casco Antiguo (Alcorcón),2.0,60 m²,NaN,Entre 60 y 80€,NaN,1.0,54 m²,Ascensor,NaN,NaN,Inmuebles de ELEPHANT REAL ESTATE,1.672615e+09,Apartamento
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15841,135000.0,Piso en venta en Calle de la Circunvalación,40.454606,-3.455234,Calle de la Circunvalación. Zona Suroeste (Tor...,3.0,67 m²,NaN,NaN,NaN,1.0,62 m²,Ascensor,NaN,A reformar,Inmuebles de ANGEL & RICARDO INMOBILIARIA,1.672960e+09,Piso
15842,210000.0,Piso en venta en Calle de Urano,40.441928,-3.473036,Calle de Urano. Zona Suroeste (Torrejón de Ardoz),1.0,79 m²,NaN,Entre 80 y 100€,NaN,2.0,63 m²,Ascensor,1,En buen estado,Inmuebles de Look & Find Retiro,1.672615e+09,Piso
15843,324600.0,Piso en venta en Fresnos,40.448403,-3.470899,Zona Suroeste (Torrejón de Ardoz),4.0,140 m²,NaN,NaN,NaN,3.0,NaN,Ascensor,1,NaN,Inmuebles de Cassas.es asesoria inmobiliaria,1.672874e+09,Piso
15844,249900.0,Piso en venta en Calle de Cibeles,40.444707,-3.473520,Calle de Cibeles. Zona Suroeste (Torrejón de A...,3.0,106 m²,NaN,Entre 80 y 100€,Entre 20 y 30 años,2.0,96 m²,NaN,1,En buen estado,Inmuebles de AS Fresnos Inmobiliaria,1.672874e+09,Piso


In [11]:
df_['location'][120]

'Ibiza, Retiro (Madrid Capital)'

In [12]:
df_ = getStreet(df_)

c:\Users\Aa\Documents\GitHub\pisoscom\parsing.py:194: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [14]:
df_['street'].unique()

array(['Abantos-Monte', 'Calle', 'Ajalvir', 'Casco', 'Bellas', 'Avenida',
       'Paseo', 'San', 'Carretera', 'Castilla,', 'Castillejos,',
       'Centro-Arroyo-La', 'Plaza', 'Embajadores-Lavapiés,', 'Centro-La',
       'Palacio,', 'Cerro-El', 'El', 'Prosperidad,', 'Almagro,',
       'Arapiles,', 'Ciudad', 'Club', 'Los', 'Pasaje', 'Concepción,',
       'Cortes-Huertas,', 'Costanilla', 'Ensanche', 'Fuentelarreina,',
       'Fuente', 'Gaztambide,', 'Golf-El', 'Goya,', 'Guindalera,',
       'Humanes', 'Ibiza,', nan, 'Jardín', 'Jerónimos,',
       'Justicia-Chueca,', 'Las', 'Centro', 'Aluche,', 'La', 'Legazpi.',
       'Lista,', 'Salamanca', 'Manzanares', 'Meco', 'Argüelles,', 'Niño',
       'Camino', 'Palomeras', 'Palos', 'Parque', 'Peñagrande,', 'Pilar,',
       'Recoletos,', 'Rejas,', 'Reyes', 'Ríos', 'Castellana,',
       'Sanchinarro,', 'Canillejas,', 'Hellín,', 'Rosas,', 'Sol,',
       'Cuatro', 'Trafalgar,', 'Universidad-Malasaña,', 'Valdeacederas,',
       'Valdelasfuentes-Norte', 